## Setup

In [ ]:
import warnings

from diffusers import (
    StableDiffusionXLPipeline,
    StableDiffusionLatentUpscalePipeline,
    EulerAncestralDiscreteScheduler,
    AutoencoderKL,
    logging,
    StableDiffusionInstructPix2PixPipeline,
    StableDiffusion3Pipeline,
)

import torch
from PIL import Image
import sys
import gc
import os

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = ""

device

In [ ]:
prompt = "berlin cathedral in a field"

loaded_prompt = (
    "concept art "
    + prompt
    + ", high quality, (magical), (nature), (futuristic), digital artwork, highly detailed"
)

loaded_prompt


In [ ]:
negative_prompt = "nsfw, cartoon, bad quality, bad anatomy, worst quality, low quality, low resolutions, extra fingers, blur, blurry, ugly, wrong proportions, watermark, image artifacts, lowres, ugly, jpeg artifacts, deformed, noisy image"

negative_prompt


## Proteus

In [ ]:
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16
)

pipe = StableDiffusionXLPipeline.from_pretrained(
    "dataautogpt3/ProteusV0.4-Lightning", vae=vae, torch_dtype=torch.float16
).to(device)

pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

In [ ]:
protoimages = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_images_per_prompt=3,
    width=1024,
    height=1024,
    guidance_scale=3.5,
    num_inference_steps=8,
).images

In [ ]:
protocomp = Image.new("RGB", (3072, 600))

x_offset = 0
for im in protoimages:
    protocomp.paste(im, (x_offset, -212))
    x_offset += im.size[0]

protocomp

# SD3

In [ ]:
pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers", torch_dtype=torch.float16, token=os.environ.get("HF_TOKEN"))
pipe = pipe.to(device)

In [ ]:
protoimages = pipe(
    loaded_prompt,
    negative_prompt=negative_prompt,
    num_images_per_prompt=3,
    width=1024,
    height=1024,
    guidance_scale=7,
    num_inference_steps=18,
).images

In [ ]:
protocomp = Image.new("RGB", (3072, 600))

x_offset = 0
for im in protoimages:
    protocomp.paste(im, (x_offset, -212))
    x_offset += im.size[0]

protocomp

In [ ]:
# 1 = left 
# 2 = middle
# 3 = right
chosenimage = 1

## Instruct pix2pix

In [ ]:
model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16, safety_checker=None
)
pipe.to(device)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

In [ ]:
prompt = "amazing, high quality, dreamlike, futuristic, colorful, vibrant"
# prompt = "make image air brushed, painted, gradients"
image = pipe(
    prompt,
    image=protoimages[chosenimage-1],
    num_inference_steps=15,
    image_guidance_scale=1,
).images[0]

image

# Garbage collect

In [ ]:
pipe = None
if device == "cuda":
    torch.cuda.empty_cache()
if device == "mps":
    torch.mps.empty_cache()
    torch.mps.current_allocated_memory()
gc.collect()

## Upscaler

In [ ]:
upscaler = StableDiffusionLatentUpscalePipeline.from_pretrained(
    "stabilityai/sd-x2-latent-upscaler", torch_dtype=torch.float16
).to(device)

In [ ]:
upscaled_image = upscaler(
    prompt=loaded_prompt,
    negative_prompt=negative_prompt,
    image=image,
    num_inference_steps=20,
    guidance_scale=0,
).images[0]

upscaled_image

# Crop

In [ ]:
cropped_image = upscaled_image.crop(
    (24, 424, 2024, 1624)
)  # from 2048/1024 to 2000/1200

cropped_image